In [1]:
import os
import sys
import torchvision
import json
import torch
from transformers.tokenization_bert import BertTokenizer
import os
from tqdm import tqdm
import math
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader

from transformers import BertModel
from transformers.tokenization_bert import BertTokenizer
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

I1220 03:33:06.521841 140323447408448 file_utils.py:39] PyTorch version 1.1.0 available.


'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from fgc_support_retri import config
from fgc_support_retri.model import BertSentenceSupModel_V2
from fgc_support_retri.utils import read_fgc, read_hotpot
from fgc_support_retri.fgc_preprocess import BertSentV2Idx, SerSentenceDataset, bert_sentV2_collate
from fgc_support_retri.eval import evalaluate_f1

In [4]:
dev_items = read_fgc(config.FGC_DEV, eval=True)

no gold supporting evidence
{'QID': 'D032Q10', 'QTYPE': '进阶题', 'QTEXT': '第二次簽訂的北美貿易協定從簽署至生效過了幾日?', 'SENTS': [{'text': '第二次签订的北美贸易协定从签署至生效过了几日?', 'start': 0, 'end': 23}], 'ANSWER': [{'ATEXT': '資訊不足無法判定', 'ATOKEN': [{'text': '资讯不足无法判定', 'start': -1}], 'ATEXT_CN': '资讯不足无法判定'}], 'ATYPE': 'Date-Duration', 'AMODE': 'Date-Duration', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '第二次签订的北美贸易协定从签署至生效过了几日?'}
no gold supporting evidence
{'QID': 'D049Q04', 'QTYPE': '申论', 'QTEXT': '「雅婷逐字稿」的命名起源為何?', 'SENTS': [{'text': '「雅婷逐字稿」的命名起源为何?', 'start': 0, 'end': 15}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「雅婷逐字稿」的命名起源为何?'}
no gold supporting evidence
{'QID': 'D086Q03', 'QTYPE': '申论', 'QTEXT': '不可再生能源的意義是什麼？', 'SENTS': [{'text': '不可再生能源的意义是什么？', 'start': 0, 'end': 13}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Sing

In [5]:
tokenizer = BertTokenizer.from_pretrained(config.BERT_EMBEDDING)
dev_set = SerSentenceDataset(dev_items, transform=torchvision.transforms.Compose([BertSentV2Idx(tokenizer)]))

I1220 03:33:10.225599 140323447408448 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00


In [6]:
dev_items[0]

{'QID': 'D004Q01',
 'SENTS': [{'text': '元祐元年（1086年），', 'start': 0, 'end': 12},
  {'text': '宋哲宗即位，', 'start': 12, 'end': 18},
  {'text': '高太皇太后垂帘听政，', 'start': 18, 'end': 28},
  {'text': '回朝任礼部郎中、中书舍人、翰林学士，', 'start': 28, 'end': 46},
  {'text': '元祐四年（1089年）拜龙图阁学士，', 'start': 46, 'end': 64},
  {'text': '曾出任杭州、颍州等知州职务，', 'start': 64, 'end': 78},
  {'text': '官至礼部尚书。', 'start': 78, 'end': 85},
  {'text': '\n绍圣元年（1094年）被哲宗贬谪至惠州、儋州（海南岛）。', 'start': 85, 'end': 114},
  {'text': '\n元符三年（1100年），', 'start': 114, 'end': 127},
  {'text': '宋徽宗即位，', 'start': 127, 'end': 133},
  {'text': '向太后垂帘听政，', 'start': 133, 'end': 141},
  {'text': '下诏让苏轼北还。', 'start': 141, 'end': 149},
  {'text': '\n建中靖国元年（1101年），', 'start': 149, 'end': 164},
  {'text': '夏天因冷饮过度，', 'start': 164, 'end': 172},
  {'text': '下痢不止，又误服黄芪，', 'start': 172, 'end': 183},
  {'text': '结果病情恶化，', 'start': 183, 'end': 190},
  {'text': '「齿间出血如蚯蚓者无数」，', 'start': 190, 'end': 203},
  {'text': '七月二十八日于常州孙氏馆病卒，', 'start': 203, 'end': 218},
  {'text': 

In [11]:
def train_sentence_model(num_epochs, batch_size, model_file_name):
    
    torch.manual_seed(12)
    bert_model_name = config.BERT_EMBEDDING
    warmup_proportion = 0.1
    learning_rate = 5e-5
    eval_frequency = 5
    
    trained_model_path = config.TRAINED_MODELS / model_file_name
    if not os.path.exists(trained_model_path):
        os.mkdir(trained_model_path)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_gpu = torch.cuda.device_count()

    model = BertSentenceSupModel_V2.from_pretrained(bert_model_name)

    model.to(device)
    if n_gpu > 1:
        model = nn.DataParallel(model)

    param_optimizer = list(model.named_parameters())

    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    
    # read data
    fgc_items = read_fgc(config.FGC_TRAIN, eval=True)
    train_items = fgc_items
    dev_items = read_fgc(config.FGC_DEV, eval=True)
    
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    train_set = SerSentenceDataset(train_items, transform=torchvision.transforms.Compose([BertSentV2Idx(tokenizer)]))
    
    dataloader_train = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=bert_sentV2_collate)
    
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    num_train_optimization_steps = int(math.ceil(len(train_set) / batch_size)) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(num_train_optimization_steps*warmup_proportion),
                                                num_training_steps=num_train_optimization_steps)
    
    print('start training ... ')
    for epoch_i in range(num_epochs+1):
        model.train()
        running_loss = 0.0
        for batch_i, batch in enumerate(tqdm(dataloader_train)):
            optimizer.zero_grad()
            
            for key in ['input_ids', 'token_type_ids', 'attention_mask', 'tf_type', 'idf_type']:
                batch[key] = batch[key].to(device)
                
            batch['label'] = batch['label'].to(dtype=torch.float, device=device)
            loss = model(batch)

            if n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu.

            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()

        print('epoch %d train_loss: %.3f' % (epoch_i, running_loss/len(dataloader_train)))

        if epoch_i % eval_frequency == 0:
            model.eval()

            with torch.no_grad():
                predictions = []
                for item in dev_items:
                    train_set = SerSentenceDataset([item], transform=torchvision.transforms.Compose([BertSentV2Idx(tokenizer)]))
                    batch = bert_sentV2_collate([sample for sample in train_set])
                    for key in ['input_ids', 'token_type_ids', 'attention_mask', 'tf_type', 'idf_type']:
                        batch[key] = batch[key].to(device)

                    prediction = model.module.predict(batch)
                    predictions.append(prediction)

                precision, recall, f1 = evalaluate_f1(dev_items, predictions)
                print('epoch %d eval_recall: %.3f eval_f1: %.3f' % (epoch_i, recall, f1))

                model_to_save = model.module if hasattr(model, 'module') else model
                torch.save(model_to_save.state_dict(),
                           str(trained_model_path / "model_epoch{0}_eval_recall_{1:.3f}_f1_{2:.3f}.m".format(epoch_i, recall, f1)))

In [ ]:
train_sentence_model(20, 18, '20191219_test1')

I1219 12:28:30.212337 140512726615872 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.0c16faba8be66db3f02805c912e4cf94d3c9cffc1f12fa1a39906f9270f76d33
I1219 12:28:30.216227 140512726615872 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "po

no gold supporting evidence
{'QID': 'D001Q11', 'QTYPE': '申论', 'QTEXT': '蘇東坡為何被後人認為是文學藝術史上的通才?', 'SENTS': [{'text': '苏东坡为何被后人认为是文学艺术史上的通才?', 'start': 0, 'end': 21}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Event', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '苏东坡为何被后人认为是文学艺术史上的通才?'}
no gold supporting evidence
{'QID': 'D006Q02', 'QTYPE': '申论', 'QTEXT': '「阿拉伯之春」運動中，走上街頭的民眾的訴求為何?', 'SENTS': [{'text': '「阿拉伯之春」运动中，', 'start': 0, 'end': 11}, {'text': '走上街头的民众的诉求为何?', 'start': 11, 'end': 24}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'ATEXT_CN': ''}], 'ATYPE': 'Object', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [], 'SHINT': [], 'QTEXT_CN': '「阿拉伯之春」运动中，走上街头的民众的诉求为何?'}
no gold supporting evidence
{'QID': 'D048Q09', 'QTYPE': '申论', 'QTEXT': '聊天機器人仰賴哪些方法讓回答愈來愈準確?', 'SENTS': [{'text': '聊天机器人仰赖哪些方法让回答愈来愈准确?', 'start': 0, 'end': 20}], 'ANSWER': [{'ATEXT': '', 'ATOKEN': [{'text': '', 'start': 0}], 'A

I1219 12:28:34.253317 140512726615872 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
  0%|          | 0/1480 [00:00<?, ?it/s]

start training ... 


100%|██████████| 1480/1480 [09:04<00:00,  2.08it/s]


...

  0%|          | 0/1480 [00:00<?, ?it/s]

epoch 16 train_loss: 0.006


  0%|          | 0/1480 [00:00<?, ?it/s]

epoch 17 train_loss: 0.004


 73%|███████▎  | 1080/1480 [06:36<02:23,  2.79it/s]

In [16]:
torch.zeros((2,1)).size(1)

1